<a href="https://colab.research.google.com/github/Devi-Prasad-G/Baseline-with-resnet-50-Limited-Data-problem-/blob/main/baseline_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_addons
import tensorflow as tf
import keras
import tensorflow_addons as tfa

## Creating a model(resnet-50)

In [ ]:
model = tf.keras.applications.ResNet50(weights=None, include_top=True, input_tensor=model_input,classes = 1000) # without using imagenet weights

In [ ]:
#we add the preprocess layers to the model instead of the data loader
input_shape = (256,256,3)
input_shape_model = (224,224,3)
s=389
inputs = tf.keras.Input(shape=input_shape)
x = tf.keras.layers.Rescaling(1.0 / 255)(inputs)
x = tf.keras.layers.Normalization(mean=[0.485, 0.456, 0.406], variance=[0.229**2, 0.224**2, 0.225**2])(x) #normalizing with mean and variance of train data
x = tf.keras.layers.RandomCrop(height=224,width=224,seed=s)(x)
outputs = model(x)
new_model = tf.keras.Model(inputs, outputs)

## Loading train data

In [ ]:
!unzip /content/drive/MyDrive/train.zip -d /content # to unzip the training file in drive

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(horizontal_flip=True,
                                  fill_mode='nearest',
                                   validation_split=0.2)
training_set = train_datagen.flow_from_directory('./train',
                                                 target_size = (256, 256),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',subset='training')

validation_set = train_datagen.flow_from_directory('./train',
                                                 target_size = (256, 256),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',subset='validation')

## model parameters and savepoints

In [ ]:
l_rate = 0.1 #original lr rate
wd = 0.0001
def scheduler(epoch, lr):
    return l_rate * (0.1 ** (epoch // 30))

callback_lr = tf.keras.callbacks.LearningRateScheduler(scheduler,verbose = 1)
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath="/content/drive/MyDrive/adv1/model_best.h5",
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

#loss = tf.keras.losses.sparse_categorical_crossentropy()
loss = tf.keras.losses.CategoricalCrossentropy()
#add weight decay to SGW
optim = tfa.optimizers.SGDW(learning_rate=l_rate, weight_decay=wd, momentum=0.9, nesterov = False, name = 'SGDW')

## Training the model

In [ ]:
new_model.compile(loss=loss,
              optimizer=optim,
              metrics=['accuracy'])
new_model.fit(training_set, validation_data = validation_set, epochs=90,steps_per_epoch=training_set.samples//32,validation_steps=validation_set.samples //32,verbose=1,callbacks = [callback_lr,checkpoint])

In [ ]:
new_model.save("/content/drive/MyDrive/adv1_final")